In [ ]:
import sys, os
import pandas as pd

In [ ]:
sys.path.insert(0,r'Y:\champ\util\pythonlib-migration\master_versions')
import Districts

In [ ]:
MODEL_RUN_DIR = r'X:\Projects\ConnectSF\2040_Baseline_v2'
OUTDIR = r'Q:\MTC\Model\ConsistencyReports\2019\Analysis\Data'
TRIPFREQ = r'8.TripTourFrequency'
TRIPLEN = r'9.TripLength'
TRIPMC = r'11.TripModeChoice'
DIST3 = os.path.join(MODEL_RUN_DIR,'DISTSIMPLE3.eqv')

In [ ]:
tazToDist, distToTaz, distToName = Districts.readDistrictsEqv(DIST3)

In [ ]:
# source: Daysim2.1 Users Guide.xlsx
modes = {0:'none',
         1:'walk',
         2:'bike',
         3:'sov',
         4:'hov 2',
         5:'hov 3',
         6:'walk to transit',
         7:'park and ride',
         8:'school bus',
         9:'other',
         }
purposes = {0:'none/home',
            1:'work',
            2:'school',
            3:'escort',
            4:'pers.bus',
            5:'shop',
            6:'meal',
            7:'social',
            8:'recreational', #(currently combined with social)
            9:'medical', #(currently combined with pers.bus.)
            10:'change mode inserted purpose',
            # extra:
            11:'work-based',
            12:'college/university',
            13:'other school'
            }

person_type = {1:'Full time worker',
               2:'Part time worker',
               3:'Non working adult age 65+',
               4:'Non working adult age<65',
               5:'University student',
               6:'High school student age 16+',
               7:'Child age 5-15',
               8:'Child age 0-4',
              }

In [ ]:
hh_cols = ['hhno','hhtaz']
pers_cols = ['hhno','pno','pptyp']
trip_cols = ['hhno','pno','tour','trip','mode','otaz','dtaz','dpurp','pathtype','travdist','id','tour_id']
tour_cols = ['hhno','pno','tour','pdpurp','tmodetp','tpathtp','parent','subtrs','tdtaz','id']

In [ ]:
hh = pd.read_csv(os.path.join(MODEL_RUN_DIR,'daysim','abm_output1','_household_2.dat'), sep=r'\s+')
pers = pd.read_csv(os.path.join(MODEL_RUN_DIR,'daysim','abm_output1','_person_2.dat'), sep=r'\s+')
df = pd.merge(hh.loc[:,hh_cols], pers.loc[:,pers_cols], on=['hhno'])
del(hh, pers)

In [ ]:
tours = pd.read_csv(os.path.join(MODEL_RUN_DIR,'daysim','abm_output1','_tour_2.dat'), sep=r'\s+')
df = pd.merge(df, tours.loc[:,tour_cols], on=['hhno','pno'])
parents = tours.loc[tours['subtrs'].gt(0)]
parents.set_index(['hhno','pno','tour'], inplace=True)
df.set_index(['hhno','pno','parent'], inplace=True)
df['parent_purp'] = parents['pdpurp']
df.reset_index(inplace=True)
del(tours)

In [ ]:
# work-based subtours
df.loc[df['parent_purp'].eq(1), 'pdpurp'] = 11
# school purpose by person type
df.loc[df['pdpurp'].eq(2) & df['pptyp'].eq(5), 'pdpurp'] = 12
df.loc[df['pdpurp'].eq(2) & df['pptyp'].isin([1,2,3,4]),'pdpurp'] = 13

In [ ]:
trips = pd.read_csv(os.path.join(MODEL_RUN_DIR,'daysim','abm_output1','_trip_2.dat'), sep=r'\s+')
df = pd.merge(df, trips.loc[:,trip_cols], on=['hhno','pno','tour'])
df['trip_mode'] = df['mode'].map(modes)
del(trips)

In [ ]:
df['district'] = df['hhtaz'].map(tazToDist)
df['district_name'] = df['district'].map(distToName)

In [ ]:
stats = (df
         .groupby(['district_name','pdpurp'])
         .agg({'tour':'size',
               'travdist':'mean'})
         .rename(columns={'tour':'trips',
                  'travdist':'average_trip_distance'})
         .reset_index()
        )

In [ ]:
tripfreq = stats.pivot(index='pdpurp', columns='district_name', values='trips')
tripfreq['purpose'] = tripfreq.index.map(purposes)

In [ ]:
(tripfreq
 .loc[:,['purpose','Downtown','Rest of San Francisco','Rest of Bay Area']]
 .to_csv(os.path.join(OUTDIR,TRIPFREQ,'tripfreq_ConnectSF_2040_Baseline.csv'))
)

In [ ]:
triplen = stats.pivot(index='pdpurp', columns='district_name', values='average_trip_distance')
triplen2 = df.groupby('pdpurp').agg({'travdist':'mean'})
triplen['Total'] = triplen2['travdist']
triplen.loc['Total',:] = df.groupby('district_name').agg('travdist').mean()
triplen.loc['Total','Total'] = df['travdist'].mean()
triplen['purpose'] = triplen.index.map(purposes)

In [ ]:
triplen

In [ ]:
(triplen
 .loc[:,['purpose','Downtown','Rest of San Francisco','Rest of Bay Area','Total']]
 .to_csv(os.path.join(OUTDIR,TRIPLEN,'triplen_ConnectSF_2040_Baseline.csv'))
)

In [ ]:
df.groupby('tmodetp').size()

In [ ]:
tripmc = df.pivot_table(index='pdpurp', columns=['mode','trip_mode'], aggfunc='size')
tripmc['purpose'] = tripfreq.index.map(purposes)
tripmc.to_csv(os.path.join(OUTDIR,TRIPMC,'tripmc_ConnectSF_2040_Baseline.csv'))